In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import csv
import datetime
import re


In [2]:
# # set here to provide data to test code

# my_date = datetime.date.today() 
# year, week_num, day_of_week = my_date.isocalendar()
# this_week = week_num

# url_for_scrape = 'https://www.metacritic.com/browse/albums/release-date/new-releases/date'
# user_agent = {'User-agent': 'Mozilla/5.0'}
# # send response
# response_score = requests.get(url_for_scrape, headers = user_agent)
# # scrape website into variable to parse
# soup_score = BeautifulSoup(response_score.text, 'html.parser')

# # create temporary lists for user scores
# userP = []
# userM = []
# userN = []
# # create/initialize dictionary 
# albums_dict = {'artist':[], 'album':[], 'date':[], 'week_num':[], 'meta_score': [], 'user_score':[]}

# soup_score.find_all('td', class_='clamp-summary-wrap')
# # create soup 
# for _ in soup_score.find_all('td', class_='clamp-summary-wrap'):
#     # scrape album name
#     albums_dict['album'].append(_.find('a', class_= 'title').text)
#     # scrape artist name and strip white space and extra characters
#     albums_dict['artist'].append(_.find('div', class_='artist').text.strip().lstrip('by '))
#     # scrape date
#     albums_dict['date'].append(_.find('div', class_='clamp-details').find('span').text)
#     # scrape meta_score, handle for changes in class name, convert data type of score to int and append to dict
#     # except set to pass since all alubms have a score

#     try:
#         albums_dict['meta_score'].append(int(_.find('div', class_='metascore_w large release positive').text))  
#     except:
#         pass
#     try:
#         albums_dict['meta_score'].append(int(_.find('div', class_='metascore_w large release mixed').text))  
#     except:
#         pass 
#     try:
#         albums_dict['meta_score'].append(int(_.find('div', class_='metascore_w large release negative').text))  
#     except:
#         pass
#     # scrape user score, handle errors for tbd/class name and append to temp list
#     try:
#         userP.append(float(_.find('div', class_='metascore_w user large release positive').text))  
#     except:
#         userP.append(0)
#     try:
#         userM.append(float(_.find('div', class_='metascore_w user large release mixed').text))  
#     except:
#         userM.append(0)
#     try:
#         userN.append(float(_.find('div', class_='metascore_w user large release negative').text))  
#     except:
#         userN.append(0)

# # merge user score by filtering scores from tbd using data type in temporary lists, convert data type of scores to int and append to dictionary        
# for a, b, c in zip(userP, userM, userN):
#     if isinstance(a, float):
#         albums_dict['user_score'].append(int(a * 10))
#     elif isinstance(b, float):
#         albums_dict['user_score'].append(int(b * 10))
#     elif isinstance(c, float):
#         albums_dict['user_score'].append(int(c * 10))
#     else:
#         albums_dict['user_score'].append(c)
# # create week_num key and values for weekly scrape
# for dates in albums_dict['date']:
#     albums_dict['week_num'].append((datetime.datetime.strptime(dates, '%B %d, %Y')).isocalendar()[1])
# # write dictionary to csv
# # create header
# fields = ['artist', 'album', 'date', 'week_num', 'meta_score', 'user_score'] 
# # create variable for data to be written
# data = zip(albums_dict['artist'], albums_dict['album'], albums_dict['date'], albums_dict['week_num'], albums_dict['meta_score'], albums_dict['user_score'])
# output_path = os.path.join('..', 'data', 'meta_scrape.csv')
# # create header
# fields = ['artist', 'album', 'date', 'week_num', 'meta_score', 'user_score'] 
# # create variable for data to be written
# data = zip(
#         albums_dict['artist'], 
#         albums_dict['album'], 
#         albums_dict['date'], 
#         albums_dict['week_num'], 
#         albums_dict['meta_score'], 
#         albums_dict['user_score']
#         )
# with open(output_path, 'w') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(fields)
#     for d in data:
#         writer.writerow(d)



In [3]:
file_path = os.path.join('..', 'data','historical_data', 'combined_csv.csv')

In [4]:
scrape_df = pd.read_csv(file_path)  
# dedup df        
# scrape_clean = scrape_df.drop_duplicates(subset = 'artist', ignore_index = True)
# scrape_clean
scrape_df

,artist,album,date,week_num,meta_score,user_score
0,The Streets,Original Pirate Material,"October 22, 2002",43,90,87
1,Tom Waits,Alice,"May 7, 2002",19,90,85
2,Queens of the Stone Age,Songs For The Deaf,"August 27, 2002",35,89,89
3,Spoon,Kill The Moonlight,"August 20, 2002",34,88,88
4,Wilco,Yankee Hotel Foxtrot,"April 23, 2002",17,87,89
...,...,...,...,...,...,...
9314,Bonny Doon,Longwave,"March 23, 2018",12,68,0
9315,A Perfect Circle,Eat the Elephant,"April 20, 2018",16,68,73
9316,Arthur Buck,Arthur Buck,"June 15, 2018",24,68,0
9317,Josh Rouse,Love in the Modern Age,"April 13, 2018",15,68,0


In [5]:
scrape_df.loc[scrape_df.index[641], 'album'] = '#N/A'

In [6]:


def create_review_url(df):
###
# takes a df latest metascrape, cleans the artist and album strings
# and adds a new column of urls to scrape review values
###
    review_urls = []
    # iterate over dataframe
    for i, j in df.iterrows():
    #     clean album and artist names of all nonalpha except for ! and accent marks
        al = re.sub(r'[^-A-Za-z0-9!áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]+', '', j['album'])
        ar = re.sub(r'[^-A-Za-z0-9!áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ ]+', '', j['artist'])
    # correct instances of multilple spaces
        al = re.sub(' +', ' ', al)
        ar = re.sub(' +', ' ', ar)
    # make lowercase and join words with - for url
        url_end = f'{al}/{ar}'.replace(" ", "-").lower() 
    # concat url for review scrape    
        url_beginning ='https://www.metacritic.com/music/'
        review_urls.append(url_beginning + url_end)
# get value of colmuns to use for new column position
    new_col_position = len(df.columns)
# Use insert to add review_urls column and values to dataframe
    df.insert(new_col_position, "review_urls", review_urls)
    return df

In [7]:
scrape_url_df = create_review_url(scrape_df)


Original Pirate Material
Alice
Songs For The Deaf
Kill The Moonlight
Yankee Hotel Foxtrot
Phrenology
The Creek Drank The Cradle
Rings Around The World
OST
Don't Give Up On Me
Didn't It Rain
Source Tags & Codes
One Beat
The Last Broadcast
Geogaddi
No!
S.T.R.E.E.T. D.A.D.
Veni Vidi Vicious
Blood Money
Yoshimi Battles The Pink Robots
The Beginning Stages Of...
We Love Life
Cobblestone Runway
In Search Of... [Second Version]
Skyscraper National Park
()
Blacklisted
Cookie: The Anthropological Mix Tape
The Rising
Come Away With Me
OOOH! (Out Of Our Heads)
Point
Folklore
Murray Street
Asleep In The Back
Under Construction
Lost Horizons
Turn On The Bright Lights
The Private Press
The Lost Tapes
Melody A.M.
God's Son
I Phantom
Something Wicked This Way Comes
Under Cold Blue Stars
Sharpen Your Teeth
The Execution of All Things
A Rush Of Blood To The Head
Yanqui U.X.O.
Electric Circus
Untouchables
Twoism [EP]
Sea Change
Nextdoorland
Live at Convocation Hall
Optometry
Italian Platinum
About A Boy 

Play What They Want
IV
Closure
Jardín
Capacity
From a Room, Vol. 1
AZD
A Fever Dream
Holiday Destination
Northern Passages
The Queen of Hearts
Salutations
Medicine Songs
You're Not As _____ As You Think
How the West Was Won
Strike a Match
World Eater
The Sounds of Crenshaw, Vol. 1
Hippopotamus
Untouchable
Max Richter: Three Worlds – Music from Woolf Works
The Animal Spirits
Out of Range
Fin
I'm a Harmony
Impressions
Borders
No Home of the Mind
War & Leisure
Somersault
Diversions Vol. 4: The Songs and Poems of Molly Drake
Sleeping Through the War
English Tapas
Crack-Up
Goin' Platinum!
Lilies
A Deeper Understanding
Rainbow
Villains
The Weather
If All I Was Was Black
Crawl Space
Paradise [EP]
Freedom Highway
Stellular
No Future
Crystal Fairy
In Between
Pleasure
Music for the Age of Miracles
The Ooz
Always Foreign
Precious Art
After the Party
Hallelujah Anyhow
Bill Orcutt
Face Your Fear
From a Room, Vol. 2
I Love You Like a Brother
The Incessant
Little Fictions
Wake in Fright
Triplicate
Re

Christine and the Queens
Loyalty
Purple
Over and Even
Ba Power
King Push - Darkest Before Dawn: The Prelude
Elaenia
Architect
New Bermuda
Bonxie
Levon Vincent
Fantasy Empire
Tomorrow Is My Turn
The Great Pretenders
Etch And Etch Deep
Constant Bop
Mr. Misunderstood
Down to Believing
Wildheart
Currents
Second Hand Heart
Asunder, Sweet and Other Distress
I Abused Animal
The Bad Plus Joshua Redman
A Fool to Care
How to Die in the North
The Epic
Y Dydd Olaf
Hinterland
Foil Deer
White Men Are Black Men Too
LP [2015]
Music in Exile
Blackbirds
High
M
Star Wars
Safe
Primrose Green
Infinite Dissolution
B'lieve I'm Goin' Down
The Miraculous
Servant of Love
Life
Panda Bear Meets the Grim Reaper
Luminiferous
Shackles Gift
Don't Lose This
Frozen Niagara Falls
Anxiety's Kiss
Shadows in the Night
The Most Important Place in the World
Male Eunuch
Me
Compton
Africa Express Presents...Terry Riley's In C Mali
Grey Tickles, Black Pressure
Piteous Gate
Sauna
Mutilator Defeated at Last
Cortar Todo
Matador
Zo

Weird Revolution
Crown Royal
Stankonia
XTRMNTR
Bachelor No. 2 (or, the last remains of the dodo)
Stories from the City, Stories from the Sea
Mass Romantic
Transcendental Blues
All Hands On The Bad One
Greatest Hits: The Evidence
Dongs of Sevotion
Supermodified
Bow Down To The Exit Sign
Mwng
Lift Your Skinny Fists Like Antennas to Heaven
Nixon
And Then Nothing Turned Itself Inside-Out
I Am Shelby Lynne
Unreasonable Behavior
Come to Where I'm From
Mermaid Avenue Vol. II
Mad for Sadness
Public Domain: Songs From the Wild Land
Discovery of a World Inside the Moone
The Moon & Antarctica
Good Looking Blues
Figure 8
The Sophtware Slump
The Noise Made By People
The Covers Record
Kid A
Music
Everything & Nothing
The "W"
Mama's Gun
Private Suit
With Ghost
Sound of Water
American III: Solitary Man
Thirteen Tales From Urban Bohemia
All That You Can't Leave Behind
In The Mode
Whoa, Nelly!
Broken By Whispers
Renegades
Rancid [2000]
Lovers Rock
The Marshall Mathers LP
Elephant Shoe
The Hour of Bewild

Skit I Allt
Hawk
Silver
If I Had A Hi-Fi
Prins Thomas
...And Then We Saw Land
Crush
The Hundred in the Hands
Mondo Cane
Pop Negro
On the Ones and Threes
Living Proof
Dear God, I Hate Myself
The Grand Theatre, Vol. 1
Love To Live
The Brutalist Bricks
Golden Archipelago
North
Two Thousand And Ten Injuries
Face Tat
The Soft Pack
History From Below
Innundir Skinni
No Ghost
One Less Heartless To Fear
In Evening Air
Tapestry Of Webs
False Priest
Home Acres
Coal Miner's Daughter: A Tribute to Loretta Lynn
Mines
Let It Sway
Pickin' Up The Pieces
Steal Your Face
Minotaur
Death Seat
Dirty Side Down
The  Long Shadow of the Paper Tiger
Volume Two
Beach Fossils
Butterfly House
Dark Night Of The Soul
Libra Scale
Bubblegum
Past Time
Church with No Magic
The Black Dirt Sessions
Earthology
Tiger Suit
Fever
All We Grow
Dream Get Together
Deth Red Sabaoth
Interpretations: The British Rock Songbook
The Drums
Magic Chairs
Wilderness Heart
Medicine County
See You On The Moon
Wu-Massacre
Blurry Blue Mountain

...

The Solution
Overgrown Path
Falling Off the Sky
La Futura
Solo Piano II
We'll Be the Moon
Write Me Back
Observator
What We Saw from the Cheap Seats
This Generation
The Minimal Wave Tapes, Vol. 2
The Bears for Lunch
The Tarnished Gold
Daybreaker
Allah-Las
Charmer
Salton Sea
Into the Waves
Provincial
Grace/Confusion
Instinct
Sunshine
Shifty Adventures in Nookie Wood
Corrosion of Conformity
The Light the Dead See
Glad All Over
Working Girl's Guitar
The Body Wins
Happy to You
A  Different Kind of Truth
Born and Raised
Area 52
Women & Work
Instrumental Tourist
The Lumineers
Dispossession
Pour Une Âme Souveraine: A Dedication to Nina Simone
The Russian Wilds
There's No Leaving Now
Apocryphon
Go Outside
Traveler
Tucson: A Country Rock Opera
Big Moon Ritual
A  Monument
Mountain Echo
Carnivale Electricos
Portico Quartet
Dream On
Orcas
Resolution
Voyageur
Tender New Signs
Leaving Eden
De La Soul's Plug 1 & Plug 2 Present...First Serve
Nothing's Gonna Change the Way You Feel About Me Now
Savane
O

To Love Is to Live
Banana Skin Shoes
Revel in the Drama
The Raging Wrath Of The Easter Bunny Demo
1000 gecs & the Tree of Clues
The Neon Skyline
Savage Mode II
Song Machine, Season One: Strange Timez
Lamb of God
Superstar
Ghosts V: Together
III
Modus Vivendi
The Ascension
Traditional Techniques
Have We Met
Giver Taker
The New OK
American Head
The Loves of Your Life
Total Freedom
La Vita Nuova
Wu Hen
Comma
After Hours
Someone New
Quickening
Sideways to New Italy
Mind Hive
X: The Godless Void and Other Stories
Bedroom
Collector
High Risk Behaviour
Red Sun Through Smoke
Never Not Together
Eight Gates
The Unraveling
Atlas Vending
Twelfth
BE
Manic
Oneiric Formulary
Welcome to Bobby's Motel
The Universal Want
Nightmare Vacation
Ghosts VI: Locusts
Nothing Is True & Everything Is Possible
Existential Reckoning
Gigaton
Viscerals
Mordechai
Chromatica
Spook the Herd
Dedicated Side B
Sorry You Couldn't Make It
Big Exercise
Stray
Self Made Man
The Godfather 3
The Allegory
The Caretaker
Just Dropped

Oh! Mighty Engine
The Quilt
Peace Queer
Forfeit/Fortune
Does You Inspire You
The Sun & The Neon Light
Electronic Projects for Musicians
All Hope Is Gone
Wrecking Ball
Come Into My House
The Formula
Entanglements
Broken Hymns, Limbs, And Skin
Too Old To Die Young
Directions To See A Ghost
A Love Extreme
19
Who Killed Harry Houdini?
Theater Of The Mind
Elephant Shell
Go Away White
When Life Gives You Lemons, You Paint That Shit Gold
Good Time
2
The Good Life
Shots
Waiting In Vain
Pebble To A Pearl
Evil Urges
Someone Else's Deja Vu
Remember [Live]
The Singles
The Way Of All Flesh
Prospekt's March [EP]
Fire Songs
Evolver
Recovery
Clipse Presents Re-Up Gang
A Hundred Million Suns
Raw Footage
The Beautiful Lie
Time Traveller
The Virgins
Mockingbird
From The Valley To The Stars
Human
Seaside Rock
Snowflake Midnight
Living On The Other Side
Diamond Hoo Ha
Mugiboogie
Surfing
Learn To Live
Breakout
Ode To J. Smith
A Picture Of The Three Of Us At The Gate To The Garden Of Eden
The Airborne Toxic 

Post Earth
South of Reality
Surviving
Gold Past Life
Everything Not Saved Will Be Lost, Pt. 1
Venus in Leo
Tomb
Sucker Punch
The  Seduction of Kansas
There Will Be No Intermission
Happy in the Hollow
Terms of Surrender
A People's History of Gauche
Black Pumas
In League with Dragons
In the Shape of a Storm
All My Relations
Stars Are the Light
Office Politics
Laughing Matter
Love Hates What You Become
Braindrops
Tasmania
It's Real
Infest the Rats' Nest
Ride Me Back Home
Look Up Sharp
In Search of the Miraculous
Desert Sessions, Vols. 11 & 12
Step 2: A Different Kind of Human
Sonic Citadel
You Deserve Love
White Stuff
Pony
Chastity Belt
Years to Burn
Networker
Mirrorland
Can You Really Find Me
Fibs
The Lion King: The Gift
Gathered
Bigger Than Life
In the End
Olympic Girls
The Route To The Harmonium
Hyperspace
Everything Not Saved Will Be Lost, Pt. 2
How Do You Love?
Highway Hypnosis
Plastic Anniversary
Shiny New Model
Turn Off the News, Build a Garden
No Words Left
Valve Bone Woe
Scholars

One for the Ghost
Always Ascending
V.
Mogic
Blood
Islands
Liberation
YSIV
Wrong Creatures
The Lillywhite Sessions
The Magic Gang
Starcrawler
Dimensional People
Shape the Future
Uncle, Duke & The Chief
Vide Noir
How to Solve Our Human Problems, Pts. 1-3
Jassbusters
Sleepless
Electric Light
Beautiful Thing
Pure Beauty
Colt
Die Lit
Stay Dangerous
so sad so sexy
One People One World
Any Day
Walk Between Worlds
Lovelaws
In the Blue Light
Icarus Falls
Felt
Indigo
Queen
Addendum
Hold On To Your Heart
Lost Friends
Pray for the Wicked
Semicircle
Masana Temples
Thank You for Today
Culture II
The Diet
The House
Cry Pretty
Warzone
I'll Be Your Girl
Darkness Rains
Sleepwalkers
White Is Relic/Irrealis Mood
Year Of The Snitch
Pinkus Abortion Technician
Woman Worldwide
Mark Kozelek
Forever Neverland
Vitriola
Caer
Between Two Shores
Longwave
Eat the Elephant
Arthur Buck
Love in the Modern Age
Dean Wareham vs. Cheval Sombre


In [ ]:
# print(scrape_df['album'].iloc[[641]]= '#N/A')
scrape_df.loc[scrape_df.index[641], 'album'] = '#N/A'

###
# found an errant / in user reviews and column is of datatype string and don't know why
# this fixes that but need to check scrape function to see why
###
# for _ in scrape_clean_url_df['user_score']:
#     if len(_) > 2:
#         _=_[:2]
#     print(_)
# print(scrape_clean_url_df['user_score'])       
# scrape_clean_url_df = int(scrape_clean_url_df['user_score'])

In [22]:
import time
def scrape_reviews(df):
    ###
    # takes a meta_scrape dataframe with review urls and 
    # scrapes the num of reviews-both critics and users-
    # album image, record label and first genre of album
    # and returns a dataframe
    ###
    img = []
    crit_num = []
    user_num = []
    label = []
    genre = []

    for i, j in df.iterrows():
        url = j['review_urls']
        user_agent = {'User-agent': 'Mozilla/5.0'}
        # send response
        response_reviews = requests.get(url, headers = user_agent)
        # scrape website into variable to parse
        soup_reviews = BeautifulSoup(response_reviews.text, 'html.parser')
        ########### comment back in from here
        # scrape album image 
        try:
            img_soup = soup_reviews.find('meta', property="og:image")
            img.append(img_soup['content'])
        except:
            img.append(None)
        # scrape num of critical reviews
        try:
            num_rev=(soup_reviews.find('span', itemprop="reviewCount"))
            crit_num.append(num_rev.text.strip())
        except:
            crit_num.append(None)
        # scrape num of user reviews
        if j['user_score'] == 0:
            user_num.append(0)
        else:
            try:
                user_revs = float(soup_reviews.find('div', class_='metascore_w user large album positive').text)
                user_num.append(int(user_revs*10))
                countpos+=1
            except:
                pass
            try:
                user_revs = float(soup_reviews.find('div', class_='metascore_w user large album mixed').text)
                user_num.append(int(user_revs*10))
                countmix+=1
            except:
                pass
            try:
                user_revs = float(soup_reviews.find('div', class_='metascore_w user large album negative').text)
                user_num.append(int(user_revs*10))
                countneg+=1
            except:
                pass
        # scrape record labels
        try:
            label_class = soup_reviews.find_all("span", itemprop="name")
            label.append(label_class[2].text.strip())
        except:
            label.append(None)
        # scrape genre
        try:
            genre.append(soup_reviews.find("span", itemprop="genre").text)
        except:
            genre.append(None)
        time.sleep(5)
    # drop column for url and add scrapped lists to dataframe
    df = df.drop(columns='review_urls').assign(album_img = img, crit_rev_num = crit_num, user_rev_num = user_num, record_label = label, album_genre = genre)
    return df



In [23]:
test_df = scrape_reviews(scrape_url_df)

ConnectionError: HTTPSConnectionPool(host='www.metacritic.com', port=443): Max retries exceeded with url: /music/restarter/torche (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fda6300ad00>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
test_df

In [ ]:
# visual of total releases by week_num

week_album_count = test_df[['week_num', 'album']].groupby(by = 'week_num').count().rename(columns={'album': 'num_albums'})
# week_num_album_count.rename(columns={'album': 'num_albums'})
week_album_count.plot.bar()

In [ ]:
genre_album_count = test_df[['album_genre', 'album']].groupby(by = 'album_genre').count().rename(columns={'album_genre': 'num_albums'})
genre_album_count.plot.bar()

In [ ]:
# get unique values for album_genre
a_list_genres = test_df['album_genre'].unique()
genre_color_dict = {}
color_num = 1
for _ in a_list_genres:
    genre_color_dict[_]=color_num
    color_num += 1
print(genre_color_dict)

# word_freq = {"Hello": 56,
#              "at": 23,
#              "test": 43,
#              "this": 43}
# new_keys = ['how', 'why', 'what', 'where']
# i = 1
# for key in new_keys:
#     word_freq[key] = i
#     i += 1
# print(word_freq)
[i for i in genre]

In [ ]:
a = [i for i in genre_color_dict.values()]
a

In [ ]:

x = test_df['crit_rev_num']
y = test_df['meta_score']

g = sns.scatterplot(x, y, hue="continent",
              data=gapminder, palette=genre_color_dict, 
                   legend='full')
g.set(xscale="log")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

groups = test_df.groupby('album_genre')
for name, group in groups:
    plt.plot(sorted(group.crit_rev_num), sorted(group.meta_score), marker='o', linestyle='', markersize=12, label=name)

plt.legend()
# test_df.groupby('album_genre')
# x = test_df['crit_rev_num']
# y = test_df['meta_score']
# colors = [i for i in genre_color_dict.values()] 
# area = test_df['user_rev_num']
# plt.scatter(x, y, s=area, c=colors, alpha=0.5)
# plt.show()

# plt.scatter( data['a'], data['b'], color=[i for i in genre_color_dict.values()] )

In [ ]:
# Making a Connection with MongoClient
client = MongoClient("mongodb://localhost:27018/")
# database
db = client["stocks_database"]
# collection
company= db["Company"]